In [1]:
!pip install pandas scikit-learn transformers

In [2]:
from google.colab import files
uploaded = files.upload()

Saving 123.csv to 123.csv


In [3]:
from google.colab import files
uploaded = files.upload()

Saving eda_123.csv to eda_123.csv


In [4]:
import pandas as pd

original_data = pd.read_csv('123.csv')
augmented_data = pd.read_csv('eda_123.csv')

print(original_data.head())
print(augmented_data.head())

   label                                               text Unnamed: 2
0      0  The plot was confusing  and the ending made no...        NaN
1      1  The customer service team was polite and very ...        NaN
2      0  I waited over an hour for my food  and it arri...        NaN
3      1  The new smartphone has a sleek design and exce...        NaN
4      0  The laptop overheats constantly and shuts down...        NaN
   label                                               text
0      1  leftover the movie was visually stunning and l...
1      1  the movie was visually stupefy and left a last...
2      1  the movie was leftover visually stunning and l...
3      1  the movie was visually stunning and left a las...
4      1  last the movie was visually stunning and left ...


In [5]:
print(original_data.columns)

Index(['label', 'text', 'Unnamed: 2'], dtype='object')


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    original_data['text'], original_data['label'], test_size=0.2, random_state=42
)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8125
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.80      0.84        10
           1       0.71      0.83      0.77         6

    accuracy                           0.81        16
   macro avg       0.80      0.82      0.81        16
weighted avg       0.82      0.81      0.81        16



In [8]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128, return_tensors="pt")

train_labels = torch.tensor(list(y_train))
test_labels = torch.tensor(list(y_test))

from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(3):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} finished with loss: {loss.item()}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 finished with loss: 0.6734016537666321
Epoch 2 finished with loss: 0.3333868980407715
Epoch 3 finished with loss: 0.1263866126537323


In [9]:
model.eval()
with torch.no_grad():
    test_inputs = test_encodings['input_ids']
    test_mask = test_encodings['attention_mask']
    outputs = model(test_inputs, attention_mask=test_mask)
    predictions = torch.argmax(outputs.logits, dim=1)
    print("BERT Classification Accuracy:", accuracy_score(test_labels, predictions))

BERT Classification Accuracy: 1.0
